# De la donnée brute à la donnée exploitable

In [1]:
import io
import re

import pandas as pd
import zipfile as zp
import numpy as np
import requests
from tqdm import tqdm


## Chargement des données brutes

In [2]:
r = requests.get("https://www.labri.fr/perso/rgiot/cours/anavis/brut.zip")
zf = zp.ZipFile(io.BytesIO(r.content), mode="a")

In [3]:
status_cols = ["date", "Station", "Status", "Nombre de vélos disponibles", "Nombre d'emplacements disponibles"]
weather_cols = ["Timestamp","Status","Clouds","Humidity","Pressure","Rain","WindGust","WindVarEnd","WindVarBeg","WindDeg","WindSpeed","Snow","TemperatureMax","TemperatureMin","TemperatureTemp"]

date_parser = lambda d: pd.datetime.strptime(d, '%Y-%m-%d %H:%M:%S')


In [4]:
summary = pd.read_csv(zf.open("brut/bicincitta_parma_summary.csv"), sep=";")
status = pd.read_csv(zf.open("brut/status_bicincitta_parma.csv"), 
                     sep=";", 
                     header=None, 
                     names=status_cols,
                     parse_dates=True,
                     date_parser=date_parser)
weather = pd.read_csv(zf.open("brut/weather_bicincitta_parma.csv"),
                     sep=";", 
                     header=None,
                     names=weather_cols,
                     parse_dates=True,
                     date_parser=date_parser)

In [8]:
summary.head(25)

,system,station,latitude,longitude,elevation
0,bicincitta_parma,01. Duc,44.807118,10.332934,51.076065
1,bicincitta_parma,02. Ospedale Maggiore,44.802263,10.306275,56.344078
2,bicincitta_parma,03. Traversetolo,44.781595,10.344492,58.324486
3,bicincitta_parma,04. Campus Chimica,44.766433,10.314547,76.587212
4,bicincitta_parma,05. Stazione FF.SS.,44.809888,10.327693,57.179089
5,bicincitta_parma,06. Ponte di Mezzo,44.802243,10.324270,57.499104
6,bicincitta_parma,07. Santa Croce,44.804064,10.314443,54.258850
7,bicincitta_parma,08. Bixio,44.796350,10.317877,59.671028
8,bicincitta_parma,09. Farini,44.795387,10.327644,61.132015
9,bicincitta_parma,10. Barilla Center,44.798416,10.340688,53.977478


In [6]:
status.head()

,date,Station,Status,Nombre de vélos disponibles,Nombre d'emplacements disponibles
0,2014-11-14 09:35:38,Duc,1,4,5
1,2014-11-14 09:35:38,Ospedale Maggiore,1,2,7
2,2014-11-14 09:35:38,Traversetolo,1,2,7
3,2014-11-14 09:35:38,Campus Chimica,1,4,5
4,2014-11-14 09:35:38,Stazione FF.SS.,1,9,10


In [7]:
weather.head()

,Timestamp,Status,Clouds,Humidity,Pressure,Rain,WindGust,WindVarEnd,WindVarBeg,WindDeg,WindSpeed,Snow,TemperatureMax,TemperatureMin,TemperatureTemp
0,2014-11-14 09:35:38,clouds,40,100,1013.0,{u'3h': 0},None,None,None,200.504,0.84,{},9.0,9.0,9.0
1,2014-11-14 09:45:05,mist,40,100,1014.0,{u'3h': 0},None,None,None,200.504,0.84,{},10.0,10.0,10.0
2,2014-11-14 09:50:05,mist,40,100,1014.0,{u'3h': 0},None,None,None,200.504,0.84,{},10.0,10.0,10.0
3,2014-11-14 09:55:05,clouds,40,100,1013.0,{u'3h': 0},None,None,None,200.504,0.84,{},9.0,9.0,9.0
4,2014-11-14 10:00:04,mist,40,100,1014.0,{u'3h': 0},None,None,None,200.504,0.84,{},10.0,10.0,10.0


## Vérification du statut

In [10]:
status = status[status.Status != 0]
status = status[status.date != None]
status.replace(to_replace=[None, "None"], value=0.0, inplace=True)

Aucune modification opérée sur les données météos. Toute les données sont valides.

## Normalisation des noms de stations

In [11]:
index = status.groupby('Station').groups.keys()

In [12]:
match_dict = {}
for i in index:
    found = False
    for station in summary['station']:
        if not(re.search(i + "$", station) is None):
            match_dict[i] = station
            found = True
    if not(found):
        match_dict[i] = 0
match_dict

{'01. Duc': '01. Duc',
 '02. Ospedale Maggiore': '02. Ospedale Maggiore',
 '03. Traversetolo': '03. Traversetolo',
 '04. Campus Chimica': '04. Campus Chimica',
 '05. Stazione FF.SS.': '05. Stazione FF.SS.',
 '06. Ponte di Mezzo': '06. Ponte di Mezzo',
 '07. Santa Croce': '07. Santa Croce',
 '08. Bixio': '08. Bixio',
 '09. Farini': '09. Farini',
 '10. Barilla Center': '10. Barilla Center',
 '11. Dus': '11. Dus',
 '12. Barezzi': '12. Barezzi',
 '13. Borgo XX Marzo': '13. Borgo XX Marzo',
 '14. Garibaldi': '14. Garibaldi',
 '15. Repubblica': '15. Repubblica',
 '16. Toschi': '16. Toschi',
 '17. Rondani': '17. Rondani',
 '18. Crocetta': '18. Crocetta',
 '19. Boito': '19. Boito',
 '20. Efsa': '20. Efsa',
 '21. Kennedy': '21. Kennedy',
 '22. Cittadella': '22. Cittadella',
 '23. Vittoria': '23. Vittoria',
 '24. Campus': '24. Campus',
 '25. Ospedale': 0,
 '25. Ospedale - viale Osacca': 0,
 '26. Palasport': 0,
 'Barezzi': '12. Barezzi',
 'Barilla Center': '10. Barilla Center',
 'Bixio': '08. Bix

In [13]:
for key, value in tqdm(match_dict.items()):
    status.loc[status.Station == key, "Station"] = value

100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [00:33<00:00,  1.53it/s]


In [14]:
status = status[status.Station != 0]

## Rééchantillonnage des données

In [15]:
resampled_status = {}

for k,v in tqdm(match_dict.items()):
    status_new = status[status['Station']==v]
    status_new.index = pd.DatetimeIndex(status_new.date)
    status_new = status_new.drop(columns=["Status","Station"])
    resampled_status[v] = status_new.resample('10min').mean()
    resampled_status[v].insert(0,'Station', v)

100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [00:38<00:00,  1.31it/s]


In [16]:
weather_dropped = weather.drop(columns=['Clouds', 'WindGust','WindVarEnd','WindVarBeg', 'TemperatureMax', 'TemperatureMin'])

In [17]:
# Resampling weather_dropped DF in order to merge with status.
weather_dropped.index = pd.DatetimeIndex(weather_dropped.Timestamp)
weather_info = weather_dropped.resample('10min').agg({'Status': ' - '.join,'Humidity':np.mean,'Pressure':np.mean,'Rain':' - '.join,'WindDeg':max,'WindSpeed':np.mean,'Snow':' - '.join,'TemperatureTemp':np.mean})
weather_info.head()

,Status,Humidity,Pressure,Rain,WindDeg,WindSpeed,Snow,TemperatureTemp
Timestamp,,,,,,,,
2014-11-14 09:30:00,clouds,100.0,1013.0,{u'3h': 0},200.504,0.84,{},9.0
2014-11-14 09:40:00,mist,100.0,1014.0,{u'3h': 0},200.504,0.84,{},10.0
2014-11-14 09:50:00,mist - clouds,100.0,1013.5,{u'3h': 0} - {u'3h': 0},200.504,0.84,{} - {},9.5
2014-11-14 10:00:00,mist - mist,100.0,1014.0,{u'3h': 0} - {u'3h': 0},200.504,0.84,{} - {},10.0
2014-11-14 10:10:00,clouds - mist,100.0,1013.5,{u'3h': 0} - {u'3h': 0},200.504,0.84,{} - {},9.5


In [18]:
# Merging status with weather.
merged_status = {}
for k,v in tqdm(resampled_status.items()):
    temp = pd.concat([resampled_status[k], weather_info], axis=1)
    merged_status[k] = temp

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [00:02<00:00,  9.16it/s]


In [19]:
merged_status["02. Ospedale Maggiore"].head()

,Station,Nombre de vélos disponibles,Nombre d'emplacements disponibles,Status,Humidity,Pressure,Rain,WindDeg,WindSpeed,Snow,TemperatureTemp
date,,,,,,,,,,,
2014-11-14 09:30:00,02. Ospedale Maggiore,2.0,7.0,clouds,100.0,1013.0,{u'3h': 0},200.504,0.84,{},9.0
2014-11-14 09:40:00,02. Ospedale Maggiore,2.0,7.0,mist,100.0,1014.0,{u'3h': 0},200.504,0.84,{},10.0
2014-11-14 09:50:00,02. Ospedale Maggiore,2.0,7.0,mist - clouds,100.0,1013.5,{u'3h': 0} - {u'3h': 0},200.504,0.84,{} - {},9.5
2014-11-14 10:00:00,02. Ospedale Maggiore,2.0,7.0,mist - mist,100.0,1014.0,{u'3h': 0} - {u'3h': 0},200.504,0.84,{} - {},10.0
2014-11-14 10:10:00,02. Ospedale Maggiore,2.0,7.0,clouds - mist,100.0,1013.5,{u'3h': 0} - {u'3h': 0},200.504,0.84,{} - {},9.5


In [ ]:
merged_status["02. Ospedale Maggiore"].head()

In [40]:
from datetime import datetime

for k,v in merged_status.items():
    
    lol = pd.date_range(start = v.index[0], end = v.index[len(v)-1], freq='10min' ).difference(v.index)
    
    # creating reference DatetimeIndex idx_ref with a minute frequency
    idx_ref = pd.DatetimeIndex(start = v.index[0], end = v.index[len(v)-1],freq='10min')

    # idx_dat represents your DatetimeIndex from the sensor
    gaps = idx_ref[~idx_ref.isin(v.index)]
    print(gaps)

/home/jules/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  


In [ ]:
import os

for k,v in tqdm(merged_status.items()):
    if not os.path.exists(f"data/{k}"):
        os.makedirs(f"data/{k}")
    v.to_csv(f"data/{k}/{k}.csv.gzip", compression='gzip')